In [1]:
import mfem.ser as mfem
import numpy as np
from bfp import *

In [2]:
def assemble_system(fes, mu, S_arr, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff, marker):

    v_coeff = VelocityCoefficient2(mu, S_arr)

    a = mfem.BilinearForm(fes)
    a.AddDomainIntegrator(mfem.ConvectionIntegrator(v_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(xs_t_coeff))
    a.AddDomainIntegrator(mfem.MassIntegrator(dS_dE_coeff))
    a.AddInteriorFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.AddBdrFaceIntegrator(mfem.TransposeIntegrator(mfem.DGTraceIntegrator(v_coeff, 1.0, -0.5)))
    a.Assemble()
    a.Finalize()
    A = a.SpMat()

    b = mfem.LinearForm(fes)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(q_coeff))
    b.AddBdrFaceIntegrator(mfem.BoundaryFlowIntegrator(inflow_coeff, v_coeff, -1.0), marker)
    b.Assemble()

    return A, b

In [ ]:
def solve_single_mu(A, b, fes, solver_type=1, rel_tol=1e-12, abs_tol=1e-12, max_iter=500):
    psi = mfem.GridFunction(fes)
    psi.Assign(1.0)

    prec = mfem.GSSmoother(A)
    solver = mfem.GMRESSolver()
    solver.SetOperator(A)
    solver.SetPreconditioner(prec)
    solver.SetRelTol(rel_tol)
    solver.SetAbsTol(abs_tol)
    solver.SetMaxIter(max_iter)
    solver.SetKDim(30)
    solver.SetPrintLevel(0)
    solver.Mult(b, psi)

    return psi, solver.GetNumIterations(), solver.GetFinalNorm()


def solve_for_multiple_mu(fes, mu_vals, w_vals, mesh, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff):
    psi_mu_list = []

    for mu, w in zip(mu_vals, w_vals):
        print("  Solving for mu =", mu)
        marker = get_marker_for_mu(mesh, mu)
        A, b = assemble_system(fes, mu, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff, marker)
        psi, iterations, final_norm = solve_single_mu(A, b, fes)

        print("    GMRES (mu =", mu, "): iterations =", iterations,
              "final norm =", final_norm)

        psi_mu_list.append((mu, w, psi))
        psi.Save("psi_mu_{:.3f}.gf".format(mu))

    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)

    for mu, w, psi in psi_mu_list:
        phi_new.Add(w, psi)

    phi_new.Save("phi_new.gf")

    return phi_new, psi_mu_list

In [ ]:
# Set parameters
nx = 3
nE = 5
x_start = 0.0
x_end = 0.3
E_start = 1
E_end = 0.01
N_ang = 4
order = 1

mesh = create_2D_mesh(nx, nE, x_start, x_end, E_start, E_end)
dim = mesh.Dimension()
fec = mfem.DG_FECollection(order, dim)
fes = mfem.FiniteElementSpace(mesh, fec)
Size = fes.GetVSize()
mu_vals, w_vals = gauss_legendre_dirs(N_ang)

#E_arr, E_grid_arr, xs_t_arr, xs_s_arr, S_arr = read_data(50)
S_arr = np.zeros(nE)
E_arr = np.linspace(E_start, E_end, nE+1)
xs_t_coeff = TotalXSCoefficient(5)
xs_s_coeff = ScatteringXSCoefficient(0)
S_coeff    = StoppingPowerCoefficient(0)
dS_dE_arr = np.zeros(nE)
dS_dE_coeff = StoppingPowerDerivativeCoefficient(0)
q_coeff = ConstantCoefficient(100)
inflow_coeff = ConstantCoefficient(0)

solve_for_multiple_mu(fes, mu_vals, w_vals, mesh, S_arr, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff)

HeatMap = HeatmapPlot(phi, mesh, fes)
HeatMap.plot_heatmap(x_start, x_end, nx, E_start, E_end, nE, vmin=10, vmax=100)

Scatter = ScatterPlot(phi, mesh, fes)
Scatter.plot_scatter()
Scatter.plot_scatter(cmap='viridis', point_size=80, vmin=30, vmax=50)

File '/Users/melekderman/github/BFP/examples1/mesh/usr/3x5_2D.mesh' already exists.
Number of unknowns: 60


: 

In [5]:
print(mesh.GetNE())  # eleman sayısını kontrol et
print(mesh.Dimension()) # dimension kontrolü


15
2


In [6]:
point = [0.1, 0.5]
print("S_coeff", S_coeff.EvalValue(point))
print("xs_t_coeff", xs_t_coeff.EvalValue(point))
print("dS_dE_coeff", dS_dE_coeff.EvalValue(point))


S_coeff 0.0
xs_t_coeff 5.0
dS_dE_coeff 0.0


AttributeError: module 'mfem.ser' has no attribute '__version__'

In [ ]:
def solve_all(fes, mesh, mu_vals, w_vals, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff):
    psi_mu_list = []

    for mu, w in zip(mu_vals, w_vals):
        print("Solving for mu =", mu)
        A, b = assemble_system(fes, mesh, mu, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff)
        psi = solve(A, b, fes)
        psi_mu_list.append((mu, w, psi))
        save_flux_mu(psi, mu)

    phi_new = mfem.GridFunction(fes)
    phi_new.Assign(0.0)

    for mu, w, psi in psi_mu_list:
        phi_new.Add(w, psi)

    phi_new.Save("phi_new.gf")

    return phi_new, psi_mu_list






# Parametreler
nx, nE = 3, 5
mesh = create_2D_mesh(nx, nE, 0, 0.3, 1.0, 0.01)
fes = mfem.FiniteElementSpace(mesh, mfem.DG_FECollection(1, 2))

mu_vals = [0.1, 0.5, 0.9]
w_vals = [0.2, 0.5, 0.3]

S_coeff = mfem.ConstantCoefficient(0.0)
xs_t_coeff = mfem.ConstantCoefficient(5.0)
dS_dE_coeff = mfem.ConstantCoefficient(0.1)
q_coeff = mfem.ConstantCoefficient(100.0)
inflow_coeff = mfem.ConstantCoefficient(0.0)

phi, psi_mu_list = solve_all(fes, mesh, mu_vals, w_vals, S_coeff, xs_t_coeff, dS_dE_coeff, q_coeff, inflow_coeff)

print("Finished!")

File '/Users/melekderman/github/BFP/examples1/mesh/usr/3x5_2D.mesh' already exists.
Solving for mu = 0.1


NameError: name 'assembler' is not defined